In [1]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv('landmarks.csv')

# Prétraitement

On décale le point 0 en (0,0) et on normalise pour que la longueur entre le point 0 et 9 soit de 1

In [38]:
# l = np.sqrt((df.x_9-df.x_0)**2 + (df.y_9-df.y_0)**2)
# for i in range(20,0,-1):
#   df['x_'+str(i)] = (df['x_'+str(i)]-df.x_0)/l
#   df['y_'+str(i)] = (df['y_'+str(i)]-df.y_0)/l

In [39]:
for i in range(1,21):
 df['d_'+str(i)] = np.sqrt((df['x_'+str(i)]-df.x_0)**2 + (df['y_'+str(i)]-df.y_0)**2)

In [40]:
l = df.d_9
for i in range(20,0,-1):
  df['d_'+str(i)] = df['d_'+str(i)]/l

In [44]:
df.head()

,Unnamed: 0,lettre,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,...,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20
0,0,0,0.246270,0.719223,0.281536,0.711615,0.322022,0.675903,0.349107,0.649566,...,0.726443,0.673381,0.921206,1.026801,0.630544,0.572396,0.831718,0.898877,0.627446,0.581415
1,1,0,0.243749,0.721589,0.277171,0.715400,0.313495,0.679614,0.344198,0.651386,...,0.703775,0.672594,0.933350,1.036892,0.641686,0.565920,0.836112,0.922656,0.658850,0.591691
2,2,0,0.246667,0.726333,0.282944,0.717739,0.317869,0.680070,0.349103,0.653151,...,0.702782,0.708060,0.886942,0.944200,0.646206,0.625241,0.749292,0.818493,0.659518,0.660635
3,3,0,0.240678,0.734765,0.278785,0.719890,0.315457,0.683802,0.348847,0.654480,...,0.761287,0.738651,0.892772,1.056187,0.703026,0.634014,0.755830,0.894208,0.708251,0.689711
4,4,0,0.237900,0.727430,0.274309,0.732956,0.317514,0.692271,0.351195,0.654238,...,0.726836,0.720784,0.925206,1.045892,0.684522,0.602392,0.835276,0.935421,0.690741,0.637738


# Méthodes de ML

In [45]:
# Pour le traitement 1
# predictor=[]
# for i in range(21):
#   predictor.append("x_"+str(i))
#   predictor.append("y_"+str(i))

# Pour le traitement 2
predictor=["d_"+str(i) for i in range(1,21) if i!=9]

target = ['lettre']

In [5]:
# X=df[predictor].values
# y = df[target].values

In [6]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [47]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [48]:
X=df[predictor]
y = df[target]

In [49]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [51]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [52]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [53]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
10/10 [==============================] - 1s 27ms/step - loss: 1.0773 - accuracy: 0.4054 - val_loss: 0.8620 - val_accuracy: 0.6533
Epoch 2/50
10/10 [==============================] - 0s 7ms/step - loss: 0.8086 - accuracy: 0.7770 - val_loss: 0.6741 - val_accuracy: 0.8667
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 0.6392 - accuracy: 0.8885 - val_loss: 0.5471 - val_accuracy: 0.9200
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - loss: 0.5118 - accuracy: 0.9426 - val_loss: 0.4450 - val_accuracy: 0.9600
Epoch 5/50
10/10 [==============================] - 0s 7ms/step - loss: 0.4082 - accuracy: 0.9730 - val_loss: 0.3570 - val_accuracy: 0.9733
Epoch 6/50
10/10 [==============================] - 0s 7ms/step - loss: 0.3231 - accuracy: 0.9831 - val_loss: 0.2798 - val_accuracy: 0.9733
Epoch 7/50
10/10 [==============================] - 0s 7ms/step - loss: 0.2539 - accuracy: 0.9831 - val_loss: 0.2141 - val_accuracy: 0.9733
Epoch 8/50
10/10 [=

In [54]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

3/3 [==============================] - 0s 5ms/step - loss: 0.0034 - accuracy: 1.0000
Loss: 0.0034185689873993397, Accuracy: 1.0


In [55]:
predictions = model.predict(X_test)

3/3 [==============================] - 0s 6ms/step


In [56]:
print(predictions)

[[9.99523461e-01 6.33417221e-05 4.13089991e-04]
 [9.99969900e-01 9.31320392e-06 2.07678386e-05]
 [9.97577429e-01 1.17897987e-03 1.24352844e-03]
 [8.92650336e-04 1.51240837e-03 9.97594833e-01]
 [4.21459445e-05 9.99902904e-01 5.48382959e-05]
 [9.38309386e-05 9.99431252e-01 4.74746514e-04]
 [1.31455145e-03 2.40127929e-03 9.96284187e-01]
 [1.27976760e-04 9.99841928e-01 3.01206655e-05]
 [9.99432206e-01 4.65637095e-05 5.21218637e-04]
 [8.60574713e-04 1.48001383e-03 9.97659385e-01]
 [2.90587777e-04 9.93488967e-01 6.22046320e-03]
 [3.61652987e-04 1.20714295e-03 9.98431206e-01]
 [9.03733773e-04 3.30052292e-03 9.95795727e-01]
 [1.29291206e-03 3.14077130e-03 9.95566368e-01]
 [3.41262406e-04 9.98863578e-01 7.95044529e-04]
 [9.99953687e-01 1.98150319e-05 2.64680730e-05]
 [1.47544651e-03 4.14873799e-03 9.94375765e-01]
 [9.99999583e-01 2.41245658e-07 8.52665707e-08]
 [2.50572242e-07 9.99999583e-01 6.68827269e-08]
 [9.99950469e-01 1.79644230e-05 3.14474964e-05]
 [9.99923289e-01 3.75067029e-05 3.906780

In [57]:
y_test

array([0, 0, 0, 2, 1, 1, 2, 1, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0, 1, 0, 0, 1,
       2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2,
       2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2, 0,
       0, 2, 0, 2, 2, 0, 1, 2, 2, 0, 2, 2, 0, 1, 1, 0, 0, 1, 2, 0, 2, 1,
       2, 0, 2, 0, 0])